# SRT Lab 1: The Sun's Brightness Temperature

In [ ]:
import digital_rf
# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from dataprocessing import SRTData

In [ ]:
# process raw SRT data to power time series
alt = np.arange(30, 51, step=5)
az = np.arange(30, 51, step=5)
srtdata = SRTData(
    datadir="ch1", params=["power", "time", "r", "phase"], start=0, end=25
).process()

## Part 1: Npoint Scan

In [ ]:
# make contour plot
fig = go.Figure()
x, y = np.asarray(alt), np.asarray(az)
z = np.asarray(srtdata["power"]).reshape((5, 5))
fig = go.Figure(data=go.Contour(z=z, x=x, y=y))
fig.update_layout(
    title="5x5 Npoint Scan",
    xaxis_title="Altitude (degrees)",
    yaxis_title="Azimuth (degrees)",
)
fig.show()

## Part 2: Azimuth/Altitude Scan

In [ ]:
# Altitude Scan
# Note to self -- look up how to convert power to temp
fig = go.Figure()
alt = np.asarray(alt)
power = np.asarray(srtdata["power"])
fig = go.Figure(data=go.Scatter(x=alt, y=power, mode="markers"))
fig.update_layout(
    title="Altitude Scan", xaxis_title="Altitude (degrees)", yaxis_title="Power",
)
fig.show()

In [ ]:
# Azimuth Scan
# Note to self -- look up how to convert power to temp
fig = go.Figure()
az = np.asarray(az)
power = np.asarray(srtdata["power"])
fig = go.Figure(data=go.Scatter(x=az, y=power, mode="markers"))
fig.update_layout(
    title="Azimuth Scan", xaxis_title="Azimuth (degrees)", yaxis_title="Power",
)
fig.show()

## Part 3: Determining Half Power Beamwidth

In [ ]:
# Polar Plot
fig = go.Figure(
    data=go.Scatterpolar(
        r=srtdata["r"], theta=srtdata["phase"] * 180 / 3.141592, mode="markers",
    )
)

fig.update_layout(showlegend=False)
fig.show()